#Imports

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import math
nltk.download('stopwords')
import builtins


import numpy as np
import pandas as pd
%load_ext google.colab.data_table
import bz2
from functools import partial
from collections import Counter, OrderedDict
import pickle
import heapq
from itertools import islice, count, groupby
from xml.etree import ElementTree
import codecs
import csv
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
import itertools
from time import time
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.



#helper functions

In [2]:
def flatten_lst(t):
    return [item for sublist in t for item in sublist]

In [3]:
def unq(list1):
 
    # initialize a null list
    unique_list = []
     
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

#getting json query data

In [5]:
import json


with open('queries_train.json', 'rt') as f:
  queries_to_docs_raw = json.load(f)

doc_lst=flatten_lst([docs for q,docs in queries_to_docs_raw.items()])


#**index from assignment 2**

In [27]:
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import os
import re
from operator import itemgetter
from pathlib import Path
import pickle
from contextlib import closing

BLOCK_SIZE = 1999998

class MultiFileWriter:
    """ Sequential binary writer to multiple files of up to BLOCK_SIZE each. """
    def __init__(self, base_dir, name):
        self._base_dir = Path(base_dir)
        self._name = name
        self._file_gen = (open(self._base_dir / f'{name}_{i:03}.bin', 'wb') 
                          for i in itertools.count())
        self._f = next(self._file_gen)
    
    def write(self, b):
      locs = []
      while len(b) > 0:
        pos = self._f.tell()
        remaining = BLOCK_SIZE - pos
        # if the current file is full, close and open a new one.
        if remaining == 0:  
          self._f.close()
          self._f = next(self._file_gen)
          pos, remaining = 0, BLOCK_SIZE
        self._f.write(b[:remaining])
        locs.append((self._f.name, pos))
        b = b[remaining:]
      return locs

    def close(self):
      self._f.close()

class MultiFileReader:
  """ Sequential binary reader of multiple files of up to BLOCK_SIZE each. """
  def __init__(self):
    self._open_files = {}

  def read(self, locs, n_bytes):
    b = []
    for f_name, offset in locs:
      if f_name not in self._open_files:
        self._open_files[f_name] = open(f_name, 'rb')
      f = self._open_files[f_name]
      f.seek(offset)
      n_read = builtins.min(n_bytes, BLOCK_SIZE - offset)
      b.append(f.read(n_read))
      n_bytes -= n_read
    return b''.join(b)
  
  def close(self):
    for f in self._open_files.values():
      f.close()

  def __exit__(self, exc_type, exc_value, traceback):
    self.close()
    return False

TUPLE_SIZE = 6       # We're going to pack the doc_id and tf values in this 
                     # many bytes.
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer

class InvertedIndex:  
  def __init__(self, docs={}):
    """ Initializes the inverted index and add documents to it (if provided).
    Parameters:
    -----------
      docs: dict mapping doc_id to list of tokens
    """
    # stores document frequency per term
    self.df = Counter()
    # length of each doc
    self.DL=Counter()
    # stores posting list per term while building the index (internally), 
    # otherwise too big to store in memory.
    self._posting_list = defaultdict(list)
    # mapping a term to posting file locations, which is a list of 
    # (file_name, offset) pairs. Since posting lists are big we are going to
    # write them to disk and just save their location in this list. We are 
    # using the MultiFileWriter helper class to write fixed-size files and store
    # for each term/posting list its list of locations. The offset represents 
    # the number of bytes from the beginning of the file where the posting list
    # starts. 
    self.posting_locs = defaultdict(list)
    


    
    for doc_id, tokens in docs.items():
      self.add_doc(doc_id, tokens)

  def add_doc(self, doc_id, tokens):
    """ Adds a document to the index with a given `doc_id` and tokens. It counts
        the tf of tokens, then update the index (in memory, no storage 
        side-effects).
    """
    w2cnt = Counter(tokens)
    self.term_total.update(w2cnt)
    for w, cnt in w2cnt.items():
      self.df[w] = self.df.get(w, 0) + 1
      self._posting_list[w].append((doc_id, cnt))

  def write_index(self, base_dir, name):
    """ Write the in-memory index to disk. Results in the file: 
        (1) `name`.pkl containing the global term stats (e.g. df).
    """
    self._write_globals(base_dir, name)

  def _write_globals(self, base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'wb') as f:
      pickle.dump(self, f)

  def __getstate__(self):
    """ Modify how the object is pickled by removing the internal posting lists
        from the object's state dictionary. 
    """
    state = self.__dict__.copy()
    del state['_posting_list']
    return state

  def posting_lists_iter(self):
    """ A generator that reads one posting list from disk and yields 
        a (word:str, [(doc_id:int, tf:int), ...]) tuple.
    """
    with closing(MultiFileReader()) as reader:
      for w, locs in self.posting_locs.items():
        b = reader.read(locs, self.df[w] * TUPLE_SIZE)
        posting_list = []
        for i in range(self.df[w]):
          doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
          tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
          posting_list.append((doc_id, tf))
        yield w, posting_list


  @staticmethod
  def read_index(base_dir, name):
    with open(Path(base_dir) / f'{name}.pkl', 'rb') as f:
      return pickle.load(f)

  @staticmethod
  def delete_index(base_dir, name):
    path_globals = Path(base_dir) / f'{name}.pkl'
    path_globals.unlink()
    for p in Path(base_dir).rglob(f'{name}_*.bin'):
      p.unlink()


  @staticmethod
  def write_a_posting_list(b_w_pl,location):
    ''' Takes a (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...]) 
    and writes it out to disk as files named {bucket_id}_XXX.bin under the 
    current directory. Returns a posting locations dictionary that maps each 
    word to the list of files and offsets that contain its posting list.
    Parameters:
    -----------
      b_w_pl: tuple
        Containing a bucket id and all (word, posting list) pairs in that bucket
        (bucket_id, [(w0, posting_list_0), (w1, posting_list_1), ...])
    Return:
      posting_locs: dict
        Posting locations for each of the words written out in this bucket.
    '''
    posting_locs = defaultdict(list)
    bucket, list_w_pl = b_w_pl

    with closing(MultiFileWriter(location, bucket)) as writer:
      for w, pl in list_w_pl: 
        # convert to bytes
        b = b''.join([(doc_id << 16 | (tf & TF_MASK)).to_bytes(TUPLE_SIZE, 'big')
                      for doc_id, tf in pl])
        # write to file(s)
        locs = writer.write(b)
      # save file locations to index
        posting_locs[w].extend(locs)
    return posting_locs

# pyspark crap and getting data from the bucket

In [7]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *


     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 62.0 MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-ba

In [8]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = SparkContext.getOrCreate(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

Copying Wikidata

In [9]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
auth.authenticate_user()

In [10]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth

project_id = 'core-period-321814'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
Copying gs://wikidata_preprocessed/multistream1_preprocessed.parquet...
| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


#getting wiki data

In [11]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
wiki_data_body = parquetFile.select("id","text").rdd.filter(lambda x: x[0] in doc_lst)
wiki_data_anchor = parquetFile.select("id","anchor_text").rdd.filter(lambda x: x[0] in doc_lst)
wiki_data_title = parquetFile.select("id","title").rdd.filter(lambda x: x[0] in doc_lst)


#tf,df and other helpers

In [12]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", #TODO: CHECK IF NEED TO ADD
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def getLenOfText(text):
  return len([token.group() for token in RE_WORD.finditer(text.lower())])
# Calc TF
# Returns a list of (token, (doc_id, tf)) pairs for each token (word) in the text
def word_count(id,text):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
 
  terms = {}
  tf = Counter()
  for token in tokens:
    if token in all_stopwords:
      continue
    current = tf.get(token, 0)
    tf[token] = current + 1
    terms[token] = (id,current +1)


  return list(terms.items())

# Returns a sorted posting list from an unsorted posting list. Sorting by tf of doc id
def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''

  return(sorted(unsorted_pl, key = lambda x: x[0]))

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda x: (x[0],len(x[1]))).partitionBy(16)

def partition_postings_and_write(postings,location):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  res = defaultdict(list)
  output_list = []
  bucketed_postings = postings.map(lambda x:(token2bucket_id(x[0]),x)).groupByKey()
  for id,content in bucketed_postings.toLocalIterator():
    #output_list.append(InvertedIndex.write_a_posting_list((id,content),"208994616")) #for gcp ########################################################
    output_list.append(InvertedIndex.write_a_posting_list((id,content),location))
  sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
  return sc.parallelize(output_list,numSlices=16)

#building the index : <font color='red'>no need</font>

> Indented block



In [29]:
from google.colab import drive
drive.mount('/content/drive')

drive_base_path='./drive/MyDrive/'

Mounted at /content/drive


In [30]:

with open(drive_base_path+"title_index/"+'title_DL.json', 'rt') as f:
  title_DL = json.load(f)
with open(drive_base_path+"anchor_index/"+'anchor_DL.json', 'rt') as f:
  anchor_DL = json.load(f)
with open(drive_base_path+"body_index/"+'body_DL.json', 'rt') as f:
  body_DL = json.load(f)



In [31]:
title_index_big=InvertedIndex.read_index(drive_base_path+'title_index', 'index')
anchor_index_big=InvertedIndex.read_index(drive_base_path+'anchor_index', 'index')
body_index_big=InvertedIndex.read_index(drive_base_path+'body_index', 'index')

# !cp drive_base_path+'body_index'

In [28]:
def tf_idf(t,d,DL):
  pls_lst=find_postings(t,index)[t]
  lst= [tf for doc,tf in pls_lst if d==doc]
  term_freq_not_normelized=lst[0]
  D=DL[d]
  tf=term_freq_not_normelized/D
  N=len(DL)
  idf=math.log2(N/index.df[t])
  return tf*idf

In [25]:
wiki_data_body.first()

Row(id=1857, text='thumb|right|On an approval ballot, the voter can select any number of candidates.\n\'\'\'Approval voting\'\'\' is an electoral system where each voter may select ("approve") any number of candidates, and the winner is the candidate approved by the largest number of voters. It is distinct from plurality voting, in which a voter may choose only one option among several, whereby the option with the most votes is chosen. It is related to score voting in which voters give each option a score on a scale, and the option with the highest total of scores is selected. Approval voting can also be used in multiwinner elections; see multiwinner approval voting.\n\nProposals to implement approval voting for municipal elections in the United States, were approved in referendums in Fargo, North Dakota, in 2018, and St. Louis, Missouri, in 2020. Fargo used approval voting in June 2020 to elect two at-large seats on its city council, and St. Louis used it to advance two candidates in 

In [26]:
def build_index_ver1(rdd_data,location,DL,big_index):
    
    word_counts = rdd_data.flatMap(lambda x: word_count(x[0], x[1],))
    DL_RDD=rdd_data.map(lambda x: (x[0],getLenOfText(x[1])))
    postings = word_counts.groupByKey().mapValues(reduce_word_counts)
    
    w2df = calculate_df(postings)
    w2df_dict = w2df.collectAsMap()
    posting_locs_list = partition_postings_and_write(postings,location).collect()

    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
      for k, v in posting_loc.items():
        super_posting_locs[k].extend(v)

    # Create inverted index instance
    inverted = InvertedIndex()
    # DL
    inverted.DL=DL
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out

    inverted.write_index(location, 'index')

    return inverted

index_body=build_index_ver1(wiki_data_body,'./body_index',body_DL)

index_title=build_index_ver1(wiki_data_title,'./title_index',title_DL)


('right', (1857, 2))
('approval', (1857, 1))


In [14]:
def word_count_anchor(id,list_of_tupels):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  t=flatten_lst([RE_WORD.finditer(text.lower()) for dest_id,text in list_of_tupels])

  tokens = [token.group() for token in t]

  terms = {}
  tf = Counter()
  for token in tokens:
    if token in all_stopwords:
      continue
    current = tf.get(token, 0)
    tf[token] = current + 1
    terms[token] = (id,current +1)


  return list(terms.items())

def build_index_ver2(rdd_data,location):
    word_counts_body = rdd_data.flatMap(lambda x: word_count_anchor(x[0], x[1]))
    postings = word_counts_body.groupByKey().mapValues(reduce_word_counts)
    DL_RDD=rdd_data.map(lambda x: (x[0],getLenOfText(x[1])))
    w2df = calculate_df(postings)

    w2df_dict = w2df.collectAsMap()
    posting_locs_list = partition_postings_and_write(postings,location).collect()

    super_posting_locs = defaultdict(list)
    for posting_loc in posting_locs_list:
      for k, v in posting_loc.items():
        super_posting_locs[k].extend(v)

    # Create inverted index instance
    inverted = InvertedIndex()
    # DL
    inverted.DL=DL_RDD.collectAsMap()
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out
    inverted.write_index(location, 'index')

    return inverted


#index_anchor=build_index_ver2(wiki_data_anchor,'./anchor_index')

In [15]:
# Indexbuilding - Change to "True" to build index
if False:
  print("Creating indices folders")
  !mkdir body_index title_index anchor_index
  print("Building body index.")
  index_body=build_index_ver1(wiki_data_body,'./body_index')
  print("Building body index done. Building title index.")
  index_title=build_index_ver1(wiki_data_title,'./title_index')
  print("Building title index done. Building anchor text index.")
  index_anchor=build_index_ver2(wiki_data_anchor,'./anchor_index')
  print("Building anchor text index done. Indexing completed.")

#queries actions


In [16]:
queries=[q for q,d in queries_to_docs_raw.items()]
q_tokenes_lst=[q.split() for q in queries]
all_terms=[item for sublist in q_tokenes_lst for item in sublist]
q_tokenes_dict={i:q for i,q in enumerate(q_tokenes_lst)}


# loading big index and DLs : <font color='green'>Indexes here</font>

loading DLs

loading indexes

Changing posting locs to match local colab file location

In [20]:
# def change_index_locs(index):
#     for term,loc_tuple in index.posting_locs.items():
#       new_list = []
#       for loc,offset in loc_tuple:
#         splited = loc.split("/")
#         for part in splited:
#           if "index" in part:
#             rebuilt = part
#           if "bin" in part:
#             rebuilt = "/content/drive/MyDrive/" + rebuilt + "/" + part
#         loc_tuple = (rebuilt,offset)
#         new_list.append(loc_tuple)
#       index.posting_locs[term] = new_list
# if True: #Change to "True" to enable
#   change_index_locs(body_index)
#   change_index_locs(anchor_index)
#   change_index_locs(title_index)




In [21]:
  # lst=list(title_index.posting_locs.values())[0]

  # with closing(MultiFileReader()) as reader:
  #   res=reader.read(lst, BLOCK_SIZE)
  # res

def find_postings(terms,index):
    res={}
    with closing(MultiFileReader()) as reader:
        for w, locs in index.posting_locs.items():
          if(res.keys()==terms):
            break
          if(w not in terms):
            continue
          else:
            b = reader.read(locs, index.df[w] * TUPLE_SIZE)
            posting_list = []
            for i in range(index.df[w]):
              doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
              tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
              posting_list.append((doc_id, tf))
            res[w]=posting_list

    return res
            

#get top n with cosine simularity on the body index

In [23]:
# works
def generate_query_tfidf_vector(query_to_search,index,DL):
    epsilon = .0000001
    total_vocab_size = len(index.df.keys())
    Q = np.zeros((total_vocab_size))
    term_vector = list(index.df.keys())    
    counter = Counter(query_to_search)
    for token in np.unique(query_to_search):
        if token.lower() in index.df.keys(): #avoid terms that do not appear in the index.               
            tf = counter[token.lower()]/len(query_to_search) # term frequency divded by the length of the query
            df = index.df[token.lower()]            
            idf = math.log((len(DL))/(df+epsilon),10) #smoothing
            
            try:
                ind = term_vector.index(token)
                Q[ind] = tf*idf
                     
            except:
                
                pass
    return Q

# works
def get_candidate_documents_and_scores(query_to_search,index,DL):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score. 
    """
    candidates = {}
    N = len(DL)   
    pls = find_postings(np.unique(query_to_search),index)

    for term in np.unique(query_to_search): 
        if term.lower() in index.df.keys():            
            list_of_doc = pls[term.lower()]                       
            normlized_tfidf = [(doc_id,(freq/DL[str(doc_id)])*math.log(N/index.df[term.lower()],10)) for doc_id, freq in list_of_doc]           
                        
            for doc_id, tfidf in normlized_tfidf:
                candidates[(doc_id,term.lower())] = candidates.get((doc_id,term.lower()), 0) + tfidf               
     
    return candidates

# works
def generate_document_tfidf_matrix(query_to_search,index,DL):
    """
    Generate a DataFrame `D` of tfidf scores for a given query. 
    Rows will be the documents candidates for a given query
    Columns will be the unique terms in the index.
    The value for a given document and term will be its tfidf score.
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    DataFrame of tfidf scores.
    """
    
    total_vocab_size = len(index.df)
    candidates_scores = get_candidate_documents_and_scores(query_to_search,index,DL) #We do not need to utilize all document. Only the docuemnts which have corrspoinding terms with the query.
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)
    D.index = unique_candidates
    D.columns = index.df.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key    
        D.loc[doc_id][term] = tfidf

    return D

from numpy import dot
from numpy.linalg import norm
def cosine_similarity(D,Q):
    """
    Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
    Generate a dictionary of cosine similarity scores 
    key: doc_id
    value: cosine similarity score
    
    Parameters:
    -----------
    D: DataFrame of tfidf scores.

    Q: vectorized query with tfidf scores
    
    Returns:
    -----------
    dictionary of cosine similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: cosine similarty score.
    """

    # YOUR CODE HERE
    # d ={}
    # mat = D.to_numpy()
   

    # for i in range(0,len(mat)):
    #   x1 = mat[i] / np.linalg.norm(mat[i])
    #   x2 = Q / np.linalg.norm(Q)

    #   val = np.matmul(x1, x2.T)
    
    #   d[i] = val


    # return d
    output = {}
    for index, row in D.iterrows():
      cos_sim = dot(row, Q)/(norm(row)*norm(Q))
      
      output[index] = cos_sim
    return output



def get_top_n(sim_dict,N=3):
    """ 
    Sort and return the highest N documents according to the cosine similarity score.
    Generate a dictionary of cosine similarity scores 
   
    Parameters:
    -----------
    sim_dict: a dictionary of similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: similarity score. We keep up to 5 digits after the decimal point. (e.g., round(score,5))

    N: Integer (how many documents to retrieve). By default N = 3
    
    Returns:
    -----------
    a ranked list of pairs (doc_id, score) in the length of N.
    """
    
    return sorted([(doc_id, builtins.round(score,5)) for doc_id, score in sim_dict.items()], key = lambda x: x[1],reverse=True)[:N]

def get_topN_score_for_queries(queries_to_search,index,DL,N=3):
    """ 
    Generate a dictionary that gathers for every query its topN score.
    
    Parameters:
    -----------
    queries_to_search: a dictionary of queries as follows: 
                                                        key: query_id
                                                        value: list of tokens.
    index:           inverted index loaded from the corresponding files.    
    N: Integer. How many documents to retrieve. This argument is passed to the topN function. By default N = 3, for the topN function. 
    
    Returns:
    -----------
    return: a dictionary of queries and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id, score). 
    """
    # YOUR CODE HERE
    d={}

    for q_id,q_terms in queries_to_search.items():
      Q=generate_query_tfidf_vector(q_terms,index,DL)
      D=generate_document_tfidf_matrix(q_terms,index,DL)
      cosim_dict=cosine_similarity(D,Q)

        
      d[q_id]=get_top_n(cosim_dict,N)

    
    return d


In [24]:

cosine_sim_result_body=get_topN_score_for_queries({i:[t.lower() for t in q_lst] for i,q_lst in q_tokenes_dict.items()},body_index,body_DL,N=99)
#cosine_sim_result_anchor=get_topN_score_for_queries({i:[t.lower() for t in q_lst] for i,q_lst in q_tokenes_dict.items()},index_body,DL_anchor)


NameError: ignored

In [ ]:
cosine_sim_result_body

# binary ranking using the title of articles and anchor text



In [ ]:
def binary_ranking_for_anchor_text(query,index):
  words,pls=zip(*index.posting_lists_iter())
  matches = Counter()
  for term in np.unique(query): 
        if term.lower() in words:            
            list_of_doc = pls[words.index(term.lower())]    
            for doc_id, freq in list_of_doc:
                matches[doc_id] = matches.get(doc_id, 0) + 1
  return sorted(matches.items(),key=lambda x:x[1], reverse =True)


In [ ]:
#Anchor text ranking
binary_ranking_for_anchor_text(q_tokenes_lst[0],index_title)

In [ ]:
#Title text ranking
binary_ranking_for_anchor_text(q_tokenes_lst[1],anchor_index)

# ranking by PageRank 

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE

  temp = pages.partitionBy(16) #partitioning for speed
  sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
  edges = temp.flatMap(lambda x:[(x[0],each[0]) for each in x[1]]).distinct() ## might have dupli
  p1 = temp.flatMap(lambda x:[(each[0],) for each in x[1]])
  p2 = temp.map(lambda x:(x[0],))
  vert = p1.union(p2).distinct()
  return edges,vert

In [ ]:
t_start = time()
#pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(wiki_data_anchor)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
#pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")


In [ ]:
pr

# ranking by article page views 

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)
# read in the counter
with open(pv_clean, 'rb') as f:
  wid2pv = pickle.loads(f.read())

#getting top n with bm25 

In [ ]:
def get_candidate_documents(query_to_search,index,words,pls):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. 
    
    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.'). 
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: generator for working with posting.
    Returns:
    -----------
    list of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score. 
    """
    candidates = []    
    for term in np.unique(query_to_search):
        if term in words:        
            current_list = (pls[words.index(term)])               
            candidates += current_list  

    return np.unique([d for d,s in candidates])


In [ ]:
import math
from itertools import chain
import time
# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
class BM25_from_index:
    """
    Best Match 25.    
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self,index,k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.N = len(DL)
        self.AVGDL = builtins.sum(DL.values())/self.N
        self.words, self.pls = zip(*self.index.posting_lists_iter())        

    def calc_idf(self,list_of_tokens):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        
        Returns:
        -----------
        idf: dictionary of idf scores. As follows: 
                                                    key: term
                                                    value: bm25 idf score
        """        
        idf = {}        
        for term in list_of_tokens:            
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass                             
        return idf
        

    def search(self, queries,N=3):
        """
        This function calculate the bm25 score for given query and document.
        We need to check only documents which are 'candidates' for a given query. 
        This function return a dictionary of scores as the following:
                                                                    key: query_id
                                                                    value: a ranked list of pairs (doc_id, score) in the length of N.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.
        
        Returns:
        -----------
        score: float, bm25 score.
        """
        # YOUR CODE HERE
        terms = []
        for query in queries:
          for term in query:
            terms.append(term)
        uniques = set(terms)
        self.idf = self.calc_idf(uniques)

        output = OrderedDict()
        for i,query in enumerate(queries):
            scores = []
            candidates = get_candidate_documents(query,self.index,self.words,self.pls)


            scores.append([(doc_id,builtins.round(self._score(query, doc_id),3)) for doc_id in (([i for i in candidates]))])
            scores = scores[0]
            scores.sort(reverse = True,key = lambda x:x[1])
            # output[' '.join(query)] = scores[:N]
            output[i] = scores[:N]
        return output

    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.
        
        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.
        
        Returns:
        -----------
        score: float, bm25 score.
        """        
        score = 0.0        
        doc_len = DL[doc_id]        
             
        for term in query:
            if term in self.index.df.keys():                
                term_frequencies = dict(self.pls[self.words.index(term)])                
                if doc_id in term_frequencies.keys():            
                    freq = term_frequencies[doc_id]
                    numerator = self.idf[term] * freq * (self.k1 + 1)
                    denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                    score += (numerator / denominator)
        return score

In [ ]:
bm25_body = BM25_from_index(index_body)
# for lst in bm25_title.pls:
#   if 1 in lst:
#     print(lst)
quety_terms_lst=[q.lower().split() for q,docs in queries_to_docs_raw.items()]
query2doc_bm25=bm25_body.search(quety_terms_lst,N=40)

# Estimate




In [ ]:
#@title Default title text
def intersection(l1,l2):      
    """
    This function perform an intersection between two lists.

    Parameters
    ----------
    l1: list of documents. Each element is a doc_id.
    l2: list of documents. Each element is a doc_id.

    Returns:
    ----------
    list with the intersection (without duplicates) of l1 and l2
    """
    return list(builtins.set(l1)&builtins.set(l2))
def precision_at_k(true_list,predicted_list,k=40):    
    """
    This function calculate the precision@k metric.

    Parameters
    -----------
    true_list: list of relevant documents. Each element is a doc_id.
    predicted_list: sorted list of documents predicted as relevant. Each element is a doc_id. Sorted is performed by relevance score
    k: integer, a number to slice the length of the predicted_list
    
    Returns:
    -----------
    float, precision@k with 3 digits after the decimal point.
    """      
    # YOUR CODE HERE

    up=len(intersection(predicted_list[:k],true_list))
    down=k
    return builtins.round(up/down,3)

def average_precision(true_list,predicted_list,k=40):
    """
    This function calculate the average_precision@k metric.(i.e., precision in every recall point).     

    Parameters
    -----------
    true_list: list of relevant documents. Each element is a doc_id.
    predicted_list: sorted list of documents predicted as relevant. Each element is a doc_id. Sorted is performed by relevance score
    k: integer, a number to slice the length of the predicted_list
    
    Returns:
    -----------
    float, average precision@k with 3 digits after the decimal point.
    """
    # YOUR CODE HERE
    # for each relevant doc that in pl and in tl activate p@k and sum them,after 
    #devide by number of relevant
    sum=0
    rel=len(intersection(predicted_list[:k],true_list))
    if rel==0:
      return 0
    for i_pl,doc in enumerate(predicted_list[:k]):
      if doc in true_list:
        sum+=precision_at_k(true_list,predicted_list[:k],i_pl+1)
    return builtins.round(sum/rel,3)

def mean_ap(true_dict,predicted_dict,k=40):
  sum=0
  for q_id,true_lst in true_dict.items():
    sum+=average_precision(true_lst,predicted_dict[q_id])
  return sum/len(true_dict)

In [ ]:
qid_to_true_docs_dict={i:d[1] for i,d in enumerate(queries_to_docs_raw.items())}
d=get_topN_score_for_queries({i:[t.lower() for t in q_lst] for i,q_lst in q_tokenes_dict.items()},index_body,40)
qid_to_returned_docs_dict={i:t[0] for i,t in d.items()}
map_cosim_body=mean_ap(qid_to_true_docs_dict,qid_to_returned_docs_dict)


bm25_dict={qid:[doc for doc,s in lst] for qid,lst in query2doc_bm25.items()}
map_bm25_body=mean_ap(qid_to_true_docs_dict,bm25_dict,40)
# average_precision(q_tokenes_dict[0],cosine_sim_result[0])

In [ ]:
print('cosine simularity strategy MAP@40 score: '+str(map_cosim_body))
print('bm25 strategy MAP@40 score: '+str(map_bm25_body))